In [2]:
import sys

sys.path.insert(0, '../lib') if '../lib'!=sys.path[0] else None
from tradealert import get_ta
ta = get_ta()

In [3]:
import pandas as pd
from tqdm.auto import tqdm
import datetime
import pathlib
import random, time

#day = datetime.datetime.strptime('1/23/2023', '%m/%d/%Y')

import pandas_market_calendars as mcal
# Create a calendar
cboe = mcal.get_calendar('CBOE_Equity_Options')

date_from = datetime.date(2023, 1, 2)
date_to = datetime.date(2023, 3, 22)#date_from#+datetime.timedelta(days=5)

try:
    ta._query('/format template=api')
    for day in cboe.schedule(date_from, date_to).index:
        day = day.to_pydatetime().date()

        trades_path = pathlib.Path(f'../data/{day.strftime("%Y%m%d")}.trades.parquet')
        oi_path = pathlib.Path(f'../data/{day.strftime("%Y%m%d")}.oi.parquet')
        top_path = pathlib.Path(f'../data/{day.strftime("%Y%m%d")}.top.parquet')

        if trades_path.exists():
            trades_df = pd.read_parquet(trades_path)
        else:
            trades_query = f'/top @all {day.strftime("%m/%d/%Y")} where dollar_impact>=100000 and size>=0.2*adv and size>=500 and open_int<size and dtx<=92 and side!=0'
            trades_df = pd.concat([
                ta.query_all_trades(f'{trades_query} and time<=\'11:00:00.000\'', clean_environment=False),
                ta.query_all_trades(f'{trades_query} and time>\'11:00:00.000\' and time<=\'13:00:00.000\'', clean_environment=False),
                ta.query_all_trades(f'{trades_query} and time>\'13:00:00.000\' and time<=\'15:00:00.000\'', clean_environment=False),
                ta.query_all_trades(f'{trades_query} and time>\'15:00:00.000\'', clean_environment=False),
            ], ignore_index=True, axis=0)
            trades_df.sort_values(by='date_time').to_parquet(trades_path)
            print(trades_path)
            time.sleep(random.randint(3, 10))

        if oi_path.exists():
            oi_df = pd.read_parquet(oi_path)
        else:
            oi_query = f'/oi {{symbol}} {day.strftime("%m/%d/%Y")}'
            oi_df = pd.concat([
                ta.query_oi(oi_query.format(symbol=symbol))
                for symbol in tqdm(trades_df.root.unique(), desc=f'OI {day}', leave=False)
            ], ignore_index=True, axis=0)
            oi_df.to_parquet(oi_path)
            print(oi_path)
            time.sleep(random.randint(3, 10))

        if top_path.exists():
            top_df = pd.read_parquet(top_path)
        else:
            #/top aapl 10/18 where time<='11:12:11.000'
            top_query = f'/top {{symbol}} {day.strftime("%m/%d/%Y")} where time<=\'{{time}}\''
            top_df = pd.concat([
                ta.query_all_trades(top_query.format(symbol=symbol, time=trades_df[(trades_df.root==symbol)].date_time.max().strftime('%H:%M:%S.%f')[:-3]), pages=1, clean_environment=False)
                for symbol in tqdm(trades_df.root.unique(), desc=f'TOP {day}', leave=False)
            ], ignore_index=True, axis=0)
            top_df.to_parquet(top_path)
            print(top_path)
            time.sleep(random.randint(3, 10))
            ta.reconnect()

finally:
    ta._query('/format template=ivan')

../data/20230322.trades.parquet


OI 2023-03-22:   0%|          | 0/77 [00:00<?, ?it/s]

../data/20230322.oi.parquet


TOP 2023-03-22:   0%|          | 0/77 [00:00<?, ?it/s]

../data/20230322.top.parquet
